<img src="img/pelf-track.png" width="80%">

# P.E.L.F.

P.E.L.F. is the **P**oint **E**ight **L**ine **F**ollower constructed from LEGO, which uses two light sensors left and right of the track (black line in the image above) to regulate its two motors (left and right). The sensor information is stored as follows:
- brightness left and right
- motor position left and right (not the speed)
- timestamp when the data is written

From these 4 data points a few other things are calculated in real time. Using geometric considerations, the x and y position is calculated. As a reference point the crossing of the track is used, as it is clearly identifiable in the brightness of the sensors (full black for both sensors). Additional information being stored is the running mode (1, 2, or 3), where 1 is the mode which stands for correctly running on the track. Modes 2 and 3 describe a state when the PELF finds itself off-track. The additional information we use in this notebook are:
- x-y position as calculated <br>
- the calculated angle <br>
- the mode of PELF (1, 2 or 3)

<img src="img/pelf-data-flow.png" width="80%">

The goal is to use the data with input and measured values (here the calculated x-y position) to arrive at methods which allow to make some statements about new data arriving. In particular, we aim to be able to:

- Tell into which direction the PELF is driving
- Identify the section of the track in which the PELF currently is

This is all under the assumption that the PELF has not left the track.

<img src="img/pelf-data-ml.png" width="80%">

# Part 1: Exploratory data analysis

Loading some standard libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

We will define the path for the raw data and the intermediate results here to use them later on.

In [ ]:
DATAPATH_RAW = "../../.assets/data/pelf/day2.pkl.zip"
DATAPATH_INTERMEDIATE = "../../.assets/data/pelf/temp/"

## Loading the data

The data is stored as a [pickle](https://docs.python.org/3/library/pickle.html) file and was collected during a busy day at the Hannover trade fair.

In [ ]:
df_raw = pd.read_pickle(DATAPATH_RAW)

In [ ]:
df_raw.head(5)

## Visualization

### The (x, y) position for all run modes

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df_raw["x_measured"],
            df_raw["y_measured"],marker=".", alpha=0.2)
plt.title("(x, y) scatter plot (all modes)",fontsize=14)
plt.ylabel("Y position",fontsize=14)
plt.xlabel("X position",fontsize=14)
plt.grid()

### Only run mode 1

In [ ]:
df_mode_1 = df_raw[df_raw["mode"]==1].copy()
df_mode_1.drop(["mode"], inplace=True, axis=1)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df_mode_1["x_measured"],
            df_mode_1["y_measured"],marker=".", alpha=0.2)
plt.title("(x, y) scatter plot (only mode 1)",fontsize=14)
plt.ylabel("Y position",fontsize=14)
plt.xlabel("X position",fontsize=14)
plt.grid()

The shape of the track is visible. Some of the lines show how the PELF is going off-course before it realizes it and changes the mode. We need to filter these out, later on.

## Calculating the motor speed

To calculate the motorspeed, we calculate the difference between two motor positions $\Delta\phi$ and divide by the difference in timestamps $\Delta t$.

In [ ]:
df_mode_1.loc[:,"motor_speed_left"] = df_mode_1["motor_position_left"].diff() / df_mode_1["timestamp"].diff()
df_mode_1.loc[:,"motor_speed_right"] = df_mode_1["motor_position_right"].diff() / df_mode_1["timestamp"].diff()

In [ ]:
df_mode_1[["motor_speed_left", "motor_speed_right", "motor_position_left", "motor_position_right", "timestamp"]].head()

## Decomposing the track into segments

To clean up the data, we now decompose the continous dataset into different tracks. A single track are the datapoints the PELF generates between a passing of the crossover section, which can be clearly identified by the two brightness sensors. At this point, the calculated x and y position is being reset to (50,50). <br>

We assume that the data set is consistent in regards of the timestamp, meaning points which happened later have a larger timestamp. A less strong assumption is that even if the temporal ordering is wrong, it is not wrong within a single round between the crossings.

In [ ]:
df_mode_1.sort_values("timestamp", inplace=True)

Instead of working with timestamps, we choose to work with the indices. This is under the assumption that the time between is each timestamp is a constant.

In [ ]:
df_mode_1.reset_index(inplace=True)

We now find all crossing points by checking for the (50,50) position. Alternatively to the calculated x and y values we could use the data points where both brightnesses indicate a line under the sensor.

In [ ]:
df_crossover = df_mode_1[(df_mode_1["x_measured"]==50) & (df_mode_1["y_measured"]==50)].copy()
df_crossover[["index", "x_measured", "y_measured"]].head(5)

Using these, we want to decompose the datasets into separate tracks. This should allow to separate the tracks which are working fine and those which are not.

### A check of the above hypothesis

We now calculate the difference in the index between the crossing points, which would correspond to the distances in time and therefore the time travelled, if the variation in motorspeed is not too big. <br>

Cases which represent a correct following of the line, but at different speed will be removed.

In [ ]:
df_crossover["index"] = df_crossover.index
df_crossover["index_diff"] = df_crossover["index"].diff()
df_crossover[["index", "index_diff"]].tail(3)

In [ ]:
plt.title("Histogram of distances between crossing points (log-scale) up to 300")
plt.hist(df_crossover["index_diff"].dropna(), bins=300)
plt.yscale("log")
plt.ylabel("# Distances")
plt.xlabel("Distances (index)")
plt.hlines(15,0,300);

We can see three main peaks (note that we are plotting y in log-scale). The first peak at the distance 0 can neglected, because everytime the PELF passes a crossover, the point (50,50) is counted twice.

Now we want to remove the data which are not lying near the two peaks around a distance of 130 and 210.
We get the bin borders and counts from the histogram above and filter them so they only include data which lies above the black bar (at y=15) seen in the histogram above are not in the first bin. We seperate the two peaks and finally cut the dataframe from above to only include the correct data.

In [ ]:
bin_counts, bin_bounds = np.histogram(df_crossover["index_diff"].dropna(), bins=300)

df_histogram = pd.DataFrame(data={"bin_counts" : bin_counts, "bin_left_bound" : bin_bounds[:-1], "bin_right_bound" : bin_bounds[1:]})
df_histogram = df_histogram[df_histogram["bin_counts"] > 15].iloc[1:]

left_bound_peak_1 = df_histogram[df_histogram["bin_left_bound"] < 200]["bin_left_bound"].min()
right_bound_peak_1 = df_histogram[df_histogram["bin_left_bound"] < 200]["bin_right_bound"].max()
left_bound_peak_2 = df_histogram[df_histogram["bin_left_bound"] > 200]["bin_left_bound"].min()
right_bound_peak_2 = df_histogram[df_histogram["bin_left_bound"] > 200]["bin_right_bound"].max()

df_seg_1 = df_crossover[(df_crossover["index_diff"]>left_bound_peak_1) & 
                                     (df_crossover["index_diff"]<right_bound_peak_1)]

df_seg_2 = df_crossover[(df_crossover["index_diff"]>left_bound_peak_2) & 
                                     (df_crossover["index_diff"]<right_bound_peak_2)]

The indices are now the index for the series when the pelf crosses the intersection and its the previous index is a certain time away. This allows for splits between left-hand and right-hand curves. Also we will add a counter, which counts how many times the PELF made the round.

In [ ]:
index_list_right = [(i, np.arange(start, stop)) for i, (start, stop) in enumerate(zip(df_seg_1["index"] - df_seg_1["index_diff"], df_seg_1["index"]))]
index_list_left = [(i, np.arange(start, stop)) for i, (start, stop) in enumerate(zip(df_seg_2["index"] - df_seg_2["index_diff"], df_seg_2["index"]))]
index_list_right_flattened = [value for array in index_list_right for value in array[1]]
index_list_left_flattened = [value for array in index_list_left for value in array[1]]

for value_right in index_list_right:
    df_mode_1.loc[value_right[1].astype(int), "nr_of_round"] = value_right[0]

for value_left in index_list_left:
    df_mode_1.loc[value_left[1].astype(int), "nr_of_round"] = value_left[0]

    df_right = df_mode_1.loc[index_list_right_flattened].drop("index", axis=1)
df_left = df_mode_1.loc[index_list_left_flattened].drop("index", axis=1)

In [ ]:
df_left["nr_of_round"].sample(5)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df_right["x_measured"],
            df_right["y_measured"],marker=".", alpha=0.2)
plt.title("Scatter plot of the right side",fontsize=14)
plt.ylabel("Y position",fontsize=14)
plt.xlabel("X position",fontsize=14)
plt.grid()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df_left["x_measured"],
            df_left["y_measured"],marker=".", alpha=0.2)
plt.title("Scatter plot of the left side",fontsize=14)
plt.ylabel("Y position",fontsize=14)
plt.xlabel("X position",fontsize=14)
plt.grid()

We can see that the right side still has irregularities: Whenever the PELF is leaving the left path at the time, which it usually takes to go through the right path, our preprocessing steps will think it is part if the right. Those false paths can be taken out by grouping the data for each track and then looking at the last data point: If it is on the left side, cut the entire track out.

In [ ]:
df_right = df_right.groupby("nr_of_round").filter(lambda x: (x.iloc[-1]["x_measured"] > 50))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df_right["x_measured"],
            df_right["y_measured"], marker=".", alpha=0.2)
plt.title("Scatter plot of the right side",fontsize=14)
plt.ylabel("Y position",fontsize=14)
plt.xlabel("X position",fontsize=14)
plt.grid()

# Saving
Let's save our intermediate results.

In [ ]:
#df_left.to_pickle(os.path.join(DATAPATH_INTERMEDIATE, "data_left_side.pkl.zip"))
#df_right.to_pickle(os.path.join(DATAPATH_INTERMEDIATE, "data_right_side.pkl.zip"))

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2024 [Point 8 GmbH](https://point-8.de)_